In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
import os
import PIL

In [2]:
with open("../Data/train.jsonl") as f:
    train_lines = f.readlines()

data_train = [eval(c) for c in train_lines]
df_train = pd.DataFrame(data_train)

with open("../Data/dev_seen.jsonl") as f:
    dev_seen_lines = f.readlines()

data_dev_seen = [eval(c) for c in dev_seen_lines]
df_dev_seen = pd.DataFrame(data_dev_seen)

with open("../Data/test_seen.jsonl") as f:
    test_seen_lines = f.readlines()

data_test_seen = [eval(c) for c in test_seen_lines]
df_test_seen = pd.DataFrame(data_test_seen)

In [ ]:
# stratified split
from sklearn.model_selection import train_test_split
df_train, df_dev_unseen = train_test_split(df_train, test_size=0.5, random_state=42, stratify=df_train['label'])


In [3]:
def load_images(df):
    images = []
    ls_exists = []
    for i in range(len(df)):
        if not os.path.exists("../Data/" + df['img'][i]):
            ls_exists.append(False)
            continue
        ls_exists.append(True)
        img = np.array(PIL.Image.open("../Data/" + df['img'][i]))
        images.append(img)
    return images, ls_exists

In [4]:
train_images, train_ls_exists = load_images(df_train)
# dev_seen_images, dev_seen_ls_exists = load_images(df_dev_seen)
# test_seen_images, test_seen_ls_exists = load_images(df_test_seen)

In [ ]:
df_train['img_exists'] = train_ls_exists
# df_dev_seen['img_exists'] = dev_seen_ls_exists
# df_test_seen['img_exists'] = test_seen_ls_exists

In [ ]:
min_length = float('inf')
min_breadth = float('inf')
max_length = 0
max_breadth = 0
for i in range(len(train_images)):
    if train_images[i].shape[0] < min_length:
        min_length = train_images[i].shape[0]
    if train_images[i].shape[1] < min_breadth:
        min_breadth = train_images[i].shape[1]
    if train_images[i].shape[0] > max_length:
        max_length = train_images[i].shape[0]
    if train_images[i].shape[1] > max_breadth:
        max_breadth = train_images[i].shape[1]
print(min_length, min_breadth, max_length, max_breadth)

94 94 823 825


In [ ]:
all_lengths = []
all_breadths = []
for i in range(len(train_images)):
    all_lengths.append(train_images[i].shape[0])
    all_breadths.append(train_images[i].shape[1])
mean_length = int(np.mean(all_lengths))
mean_breadth = int(np.mean(all_breadths))
print(mean_length, mean_breadth)

525 597


In [ ]:
def resize_images(images, length, breadth):
    resized_images = []
    for i in range(len(images)):
        image_length = images[i].shape[0]
        image_breadth = images[i].shape[1]
        image_new = images[i]
        if image_length > length:
            image_new = image_new[:length, :, :]
        elif image_length < length:
            # repeat the image horizontally
            extra = length - image_length
            if extra < image_length:
                chunk = image_new[image_length - extra:, :, :]
            else:
                number_of_full_chunks = int(extra / image_length)
                remainder = extra % image_length
                chunk = image_new
                for j in range(number_of_full_chunks-1):
                    chunk = np.concatenate((chunk, image_new), axis=0)
                if remainder > 0:
                    chunk = np.concatenate((chunk, image_new[:remainder, :, :]), axis=0)
            image_new = np.concatenate((image_new, chunk), axis=0)
        if image_breadth > breadth:
            image_new = image_new[:, :breadth, :]
        elif image_breadth < breadth:
            # repeat the image vertically
            extra = breadth - image_breadth
            if extra < image_breadth:
                chunk = image_new[:, image_breadth - extra:, :]
            else:
                number_of_full_chunks = int(extra / image_breadth)
                remainder = extra % image_breadth
                chunk = image_new
                for j in range(number_of_full_chunks-1):
                    chunk = np.concatenate((chunk, image_new), axis=1)
                if remainder > 0:
                    chunk = np.concatenate((chunk, image_new[:, :remainder, :]), axis=1)
            image_new = np.concatenate((image_new, chunk), axis=1)
        resized_images.append(image_new)
    return resized_images

In [ ]:
resized_train_images = resize_images(train_images, mean_length, mean_breadth)

In [ ]:
# find image smaller than mean_length and mean_breadth
for i in range(len(train_images)):
    if train_images[i].shape[0] < mean_length or train_images[i].shape[1] < mean_breadth:
        print(train_images[i].shape)
        # plt.imshow(train_images[i])
        plt.imshow(resized_train_images[i])
        break


(400, 265, 3)
